In [1]:
import pandas as pd
df = pd.read_csv('/Users/niharikarawat/Documents/VSCODE/airbnb_price_estimator/B-merged_airbnb.csv')

In [2]:
print(df.columns)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license', 'city'],
      dtype='object')


In [3]:
df = df.drop(columns = ['id', 'name', 'host_name', 'neighbourhood_group','last_review', 'reviews_per_month', 'number_of_reviews_ltm', 'license', 'calculated_host_listings_count'])

In [4]:
df['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'],
      dtype=object)

In [5]:
df = df.dropna(subset=['price'])

In [6]:
print(df.describe())
print(df.info())

            host_id      latitude     longitude         price  minimum_nights  \
count  6.566200e+04  65662.000000  65662.000000  65662.000000    65662.000000   
mean   2.070268e+08     34.763199   -119.159932    383.024778       14.843319   
std    2.085335e+08      1.751923      1.900529   2151.683891       24.209843   
min    2.900000e+01     32.540760   -122.519040      7.000000        1.000000   
25%    2.548342e+07     33.935310   -121.868670    100.000000        2.000000   
50%    1.131378e+08     34.081220   -118.361750    167.000000        3.000000   
75%    3.944767e+08     36.975095   -118.126478    289.000000       30.000000   
max    7.019628e+08     37.881620   -116.936852  54280.000000      730.000000   

       number_of_reviews  availability_365  
count       65662.000000      65662.000000  
mean           51.351802        234.444367  
std           100.631192        111.146222  
min             0.000000          0.000000  
25%             1.000000        147.000000  


In [7]:
price_bounds = {
    'Entire home/apt': {'min': 150, 'max': 5000},
    'Private room':    {'min': 100, 'max': 500},
    'Shared room':     {'min': 50, 'max': 250},
    'Hotel room':      {'min': 100, 'max': 5000}
}

def is_price_valid(row):
    bounds = price_bounds[row['room_type']]
    return bounds['min'] <= row['price'] <= bounds['max']

df = df[df.apply(is_price_valid, axis=1)]

In [8]:
min_nights_bounds = {
    'min': 1, 'max' : 700
}

def clip_min_nights(row):
    bounds = min_nights_bounds
    return max(bounds['min'], min(row['minimum_nights'], bounds['max']))

df['minimum_nights'] = df.apply(clip_min_nights, axis=1)

In [9]:
cap = df['number_of_reviews'].quantile(0.99)
print(cap)
df['number_of_reviews'] = df['number_of_reviews'].clip(upper=cap)

448.0


In [10]:
df = df[
    (df['latitude'] >= 32.5) & (df['latitude'] <= 42.0) &
    (df['longitude'] >= -122.5) & (df['longitude'] <= -117.0)
]


In [11]:
df.to_csv('/Users/niharikarawat/Documents/VSCODE/airbnb_price_estimator/D-cleaned_airbnb.csv', index=False)